# Control the Franka Panda Robot via Teleoperation.


* Control the gripper's pose using the xbox joystick
* Guide the robot's movement through an **OSC** controller
* Implement recording and replay functionality to store and replay control inputs, used for demonstration and imitation learning training


## Testing Xbox Controller Input

* Run a test program to confirm that the controller is receiving input signals
* Press the corresponding buttons to print information

In [ ]:
import os
import sys
import time

current_file_path = os.path.abspath('')
project_root = os.path.dirname(current_file_path)

if project_root not in sys.path:
    sys.path.append(project_root)

from orca_gym.devices.xbox_joystick import XboxJoystickManager

def test_xbox_controller():
    try:
        joystick_manager = XboxJoystickManager()
        joystick_names = joystick_manager.get_joystick_names()
        joysticks = [joystick_manager.get_joystick(name) for name in joystick_names]
        while True:
            joystick_manager.update()      
            for i, joystick in enumerate(joysticks):
                state = joystick.get_state()
                print("Controller:", joystick_names[i])
                print("Buttons:", state["buttons"])
                print("Axes:", state["axes"])
                print("Hats:", state["hats"])
            time.sleep(1)
    except KeyboardInterrupt:
        joystick_manager.close()
        print("Controller closed")

if __name__ == '__main__':
    test_xbox_controller()


## Testing Keyboard Input

To accommodate different operating systems, the Pygame library is used to implement keyboard input detection. When running the program, a small window will appear, and you need to focus on this window for key inputs to be captured. Press the corresponding keys (A/W/S/D/Up/Down, etc.) and observe the printed output.

In [ ]:
import os
import sys
import time

current_file_path = os.path.abspath('')
project_root = os.path.dirname(current_file_path)

if project_root not in sys.path:
    sys.path.append(project_root)

from orca_gym.devices.keyboard import KeyboardInput

def test_keyboard_controller():
    try:
        controller = KeyboardInput()
        while True:
            controller.update()            
            state = controller.get_state()
            print("Keys:", state)
            if state["Esc"]:
                raise KeyboardInterrupt("Esc key pressed")
                break
            time.sleep(1)
    except KeyboardInterrupt:
        controller.close()
        print("Controller closed")

if __name__ == '__main__':
    test_keyboard_controller()

In the example above, a keyboard class instance corresponds to a control program. If you run multiple environments in parallel, each with a robot that can be controlled, you will need to switch between different control instances (small windows).

If you want to use a single keyboard instance to control multiple environments simultaneously, such as operating a formation of robots using keypresses, you can use the `KeyboardServer` and `KeyboardClient` objects.

Since Jupyter notebooks are not convenient for demonstrating multi-process examples, this example is located in `examples/teleoperation/keyboard_input`.

* Run `run_server.py` to start a keyboard control service that listens for user input. A small window will pop up.
* Run `run_clients.py` to start two clients that listen to the server's broadcasts. At this point, focus on the small window that popped up earlier, and when you press a key, both clients will print the received keypress input.


## Control the Franka Panda Robot Arm

Apply offsets to the mocap point inputs based on the controller state

* **Using Xbox Controller:**
    * Left joystick controls parallel movement of the gripper
    * Right joystick controls pitch and yaw of the gripper
    * LT/RT controls forward and backward extension of the gripper
    * LB/RB controls rotation of the gripper
    * A/B controls opening and closing of the gripper

 In this example, The player's task is to operate the robot arm to pick up the block and lift the block vertically up to 0.1 meters. The operations will be recorded and stored in a .hdh5 file (refer to the implementation of the `run_example` function). In the next step, we will replay these operations.

 The gameplay is divided into multiple episodes. At the end of each episode, the game will pause and ask the player whether to save the record for this episode. Enter "Y" and press Enter to keep the record, or enter "N" and press Enter to discard it. After exiting the program, you can view the HDF5 file (it's recommended to use the H5Web plugin in VS Code) to see the data tracking for each episode.

In [ ]:
import os
import sys
import time

current_file_path = os.path.abspath('')
project_root = os.path.dirname(current_file_path)

if project_root not in sys.path:
    sys.path.append(project_root)


import gymnasium as gym
from gymnasium.envs.registration import register
from datetime import datetime
from envs.orca_gym_env import RewardType
from envs.robomimic.robomimic_env import ControlType
from envs.robomimic.dataset_util import DatasetWriter

import numpy as np
import asyncio
import nest_asyncio
nest_asyncio.apply()

TIME_STEP = 0.01
MAX_EPISODE_STEPS = 10 / TIME_STEP # 10 seconds in normal speed.

def register_env(orcagym_addr, env_name, env_index, **kwargs) -> str:
    orcagym_addr_str = orcagym_addr.replace(":", "-")
    env_id = env_name + "-OrcaGym-" + orcagym_addr_str + f"-{env_index:03d}"
    gym.register(
        id=env_id,
        entry_point="envs.franka_control.franka_teleoperation_env:FrankaTeleoperationEnv",
        kwargs=kwargs,
        max_episode_steps= MAX_EPISODE_STEPS,  # 10 seconds
        reward_threshold=0.0,
    )
    return env_id

def run_episode(env, dataset_writer):
    obs, info = env.reset(seed=42)
    obs_list = {obs_key: list([]) for obs_key, obs_data in obs.items()}
    reward_list = []
    done_list = []
    info_list = []    
    terminated_times = 0
    while True:
        start_time = datetime.now()

        action = env.action_space.sample()
        obs, reward, terminated, truncated, info = env.step(action)
        
        for obs_key, obs_data in obs.items():
            obs_list[obs_key].append(obs_data)
            
        reward_list.append(reward)
        done_list.append(0 if not terminated else 1)
        info_list.append(info)
        terminated_times = terminated_times + 1 if terminated else 0

        if terminated_times >= 5 or truncated:
            return obs_list, reward_list, done_list, info_list

        elapsed_time = datetime.now() - start_time
        if elapsed_time.total_seconds() < TIME_STEP:
            time.sleep(TIME_STEP - elapsed_time.total_seconds())
        else:
            print("Over time! elapsed_time (ms): ", elapsed_time.total_seconds() * 1000)

def user_comfirm_save_record(task_result):
    while True:
        user_input = input(f"Task is {task_result}! Do you want to save the record? (y/n): ")
        if user_input == 'y':
            return True
        elif user_input == 'n':
            return False
        else:
            print("Invalid input! Please input 'y' or 'n'.")

def do_teleoperation(env, dataset_writer):
    while True:
        obs_list, reward_list, done_list, info_list = run_episode(env, dataset_writer)
        task_result = "Success" if done_list[-1] == 1 else "Failed"
        save_record = user_comfirm_save_record(task_result)
        if save_record:
            dataset_writer.add_demo({
                'states': np.array([np.concatenate([info["state"]["qpos"], info["state"]["qvel"]]) for info in info_list]),
                'actions': np.array([info["action"] for info in info_list]),
                'rewards': np.array(reward_list),
                'dones': np.array(done_list),
                'obs': obs_list
            })

async def run_example():
    try:
        orcagym_addr = "localhost:50051"
        print("simulation running... , orcagym_addr: ", orcagym_addr)

        env_name = "Franka-Teleoperation-v0"
        env_index = 0
        kwargs = {'frame_skip': 1,   
                    'reward_type': RewardType.SPARSE,
                    'orcagym_addr': orcagym_addr, 
                    'agent_names': ['Panda'], 
                    'time_step': TIME_STEP,
                    'control_type': ControlType.TELEOPERATION,
                    'control_freq': 20}        
        env_id = register_env(orcagym_addr, env_name, env_index, **kwargs)
        print("Registered environment: ", env_id)

        env = gym.make(env_id)        
        print("Starting simulation...")

        formatted_now = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        dataset_writer = DatasetWriter(file_path=f"teleoperation_dataset_{formatted_now}.hdf5",
                                       env_name=env_id,
                                       env_version=env.get_env_version(),
                                       env_kwargs=kwargs)

        do_teleoperation(env, dataset_writer)
        dataset_writer.finalize()
    except KeyboardInterrupt:
        print("Simulation stopped")        
        dataset_writer.finalize()
        env.close()
    

if __name__ == "__main__":
    """
    An example of an OSC (Operational Space Control) motion algorithm controlling a Franka robotic arm.
    Level: Franka_Joystick
    Differences from the mocap version:
    1. Motor control uses torque output (moto) instead of setting joint angles.
    2. Torque calculation is based on the OSC algorithm.
    3. The mocap point can move freely and is not welded to the site; the pulling method is not used.
    """
    asyncio.run(run_example())


## Replay the Previous Operations
